In [105]:
# Import pandas, numpy and the dataset, save it in a object called 'sentiment'
# Your code here
import numpy as np
import pandas as pd
submission = pd.read_csv('./data/sampleSubmission.csv', encoding='latin-1')
# Let's check sentiment.head(10) and sample(10)
# Your code here
submission.head(20)
submission.sample(10)
#len(submission)

,id,sentiment
7308,7336_7,0
16842,6186_4,0
17139,2114_3,0
13055,5003_1,0
1167,12273_1,0
15322,10692_4,0
15718,11946_7,0
4610,3534_3,0
22799,11136_10,0
21745,1854_10,0


In [132]:
trainData = pd.read_table('./data/labeledTrainData.tsv')
testData = pd.read_table('./data/testData.tsv')
trainData[['sentiment','review']].head()


,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [118]:
finalTest = pd.merge(submission, testData )
finalTest.head()
#len(submission)

,id,sentiment,review
0,12311_10,0,Naturally in a film who's main themes are of m...
1,8348_2,0,This movie is a disaster within a disaster fil...
2,5828_4,0,"All in all, this is a movie for kids. We saw i..."
3,7186_2,0,Afraid of the Dark left me with the impression...
4,12128_7,0,A very accurate depiction of small time mob li...


In [119]:
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report # for the check the error and accuracy of the model
import warnings
warnings.filterwarnings('ignore')

X_train = trainData['review']
y_train = trainData['sentiment']
X_test = finalTest['review']
y_test = finalTest['sentiment']


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)# in this our main data is splitted into train and test

# we can check their dimension
print(X_test.shape)
print(y_test.shape)




(25000,)
(25000,)


In [120]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [121]:
import re
def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

# Create some random texts for testing the function preprocessor()
print(preprocessor(''))



In [122]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

# write a function called `tokenizer()` that split a text into list of words
# Your code here
def tokenizer(text):
    token=preprocessor(text)
    return token.split()

def tokenizer_porter(text):
    tokens = tokenizer(text)
    stem_words = []
    for word in tokens:
        word=porter.stem(word)   
        stem_words.append(word)
    return stem_words 

In [123]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [124]:

import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here

clf.fit(X_train, y_train) 

# Make predictions on testing set
predictions = clf.predict(X_test)


print('Accuracy:', accuracy_score(y_test, predictions))
cm = confusion_matrix(y_test, predictions)
print(cm)
# heatmap = sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu"))
# print(heatmap)

Accuracy: 0.49916
[[12479 12521]
 [    0     0]]


In [136]:
twits = [
    "This movie is a disaster",
    "I couldn't be happier!"
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Positive, Negative = {preds[i]}')

This movie is a disaster --> Positive, Negative = [0.76415696 0.23584304]
I couldn't be happier! --> Positive, Negative = [0.47156403 0.52843597]


In [139]:
import pickle
import os

pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)